In [1]:
import json
import numpy as np
import pandas as pd
import csv

In [2]:
df_reviews = pd.read_json('./data/IMDB_reviews.json', lines=True).\
             drop_duplicates('review_text').sample(frac=1)
df_reviews.review_date = pd.to_datetime(df_reviews.review_date, infer_datetime_format=True)
df_reviews.user_id = df_reviews.user_id.astype('category')
print(df_reviews.shape)
df_reviews.tail()

(573385, 7)


,is_spoiler,movie_id,rating,review_date,review_summary,review_text,user_id
156473,True,tt1276104,10,2017-03-30,"A well-acted, intriguing film.","Film noir meets sci-fi as great acting, though...",ur43650030
93770,False,tt0112471,8,2000-10-05,Once Again Linklater Avoids Mediocrity!,"As in his other movies, Linklater has a terrif...",ur0443283
474003,False,tt1013752,6,2014-01-14,A good reboot to the franchise.,The Fast and the Furious series got really wea...,ur22618683
235726,False,tt2126355,7,2015-05-29,Surprisingly Good Disaster Movie,I went in to this with admittedly very low exp...,ur4844673
485562,False,tt0105643,1,2006-08-03,Quite probably the worst theatrically released...,"I don't even know what this was, it was that b...",ur4195782


In [3]:
# only less than 20% of the user comments' explicitly say that they contain spoilers
df_reviews.review_text.str.contains('spoiler', case=False).sum()/df_reviews.is_spoiler.sum()

0.17516041788195366

In [4]:
# imdb information about the movies
df_movies = pd.read_json('./data/IMDB_movie_details.json', lines=True)
df_movies.release_date = pd.to_datetime(df_movies.release_date, infer_datetime_format=True)
print(df_movies.shape)
df_movies.tail()

(1572, 7)


,duration,genre,movie_id,plot_summary,plot_synopsis,rating,release_date
1567,1h 53min,"[Sci-Fi, Thriller]",tt0289879,Evan Treborn grows up in a small town with his...,"In the year 1998, Evan Treborn (Ashton Kutcher...",7.7,2004-01-23
1568,1h 41min,[Drama],tt1723811,Brandon is a 30-something man living in New Yo...,"Brandon (Michael Fassbender) is a successful, ...",7.2,2012-01-13
1569,1h 46min,"[Action, Drama, History]",tt5013056,Evacuation of Allied soldiers from the British...,The film alternates between three different pe...,8.1,2017-07-21
1570,1h 33min,"[Comedy, Drama]",tt0104014/,"For a while now, beautiful 24-year-old Diana B...",,5.3,1992-02-21
1571,1h 32min,"[Drama, Thriller]",tt0114142/,"The marriage of David Burgess, a senior execut...",,4.0,1999-01-29


In [9]:
# join the two dataframe according to movie id
df_reviews = df_reviews.merge(df_movies, on="movie_id",how="left",suffixes=('_review','_movie'))

,is_spoiler,movie_id,rating_review,review_date,review_summary,review_text,user_id,duration_review,genre_review,plot_summary_review,...,plot_summary_movie,plot_synopsis_movie,rating_review,release_date_movie,duration,genre,plot_summary,plot_synopsis,rating_movie,release_date
573380,True,tt1276104,10,2017-03-30,"A well-acted, intriguing film.","Film noir meets sci-fi as great acting, though...",ur43650030,1h 53min,"[Action, Crime, Drama]","Joe is classified as a ""looper"", a job in whic...",...,"Joe is classified as a ""looper"", a job in whic...","In 2044, the United States has suffered econom...",7.4,2012-09-28,1h 53min,"[Action, Crime, Drama]","Joe is classified as a ""looper"", a job in whic...","In 2044, the United States has suffered econom...",7.4,2012-09-28
573381,False,tt0112471,8,2000-10-05,Once Again Linklater Avoids Mediocrity!,"As in his other movies, Linklater has a terrif...",ur0443283,1h 41min,"[Drama, Romance]",American tourist Jesse and French student Celi...,...,American tourist Jesse and French student Celi...,The film starts with Jesse meeting Céline on a...,8.1,1995-01-27,1h 41min,"[Drama, Romance]",American tourist Jesse and French student Celi...,The film starts with Jesse meeting Céline on a...,8.1,1995-01-27
573382,False,tt1013752,6,2014-01-14,A good reboot to the franchise.,The Fast and the Furious series got really wea...,ur22618683,1h 47min,"[Action, Crime, Thriller]",Heading back to the streets where it all began...,...,Heading back to the streets where it all began...,This fourth 'Fast and the Furious' film begins...,6.6,2009-04-03,1h 47min,"[Action, Crime, Thriller]",Heading back to the streets where it all began...,This fourth 'Fast and the Furious' film begins...,6.6,2009-04-03
573383,False,tt2126355,7,2015-05-29,Surprisingly Good Disaster Movie,I went in to this with admittedly very low exp...,ur4844673,1h 54min,"[Action, Adventure, Drama]","In San Andreas, California is experiencing a s...",...,"In San Andreas, California is experiencing a s...","The film opens with the girl, Natalie (Morgan ...",6.1,2015-05-29,1h 54min,"[Action, Adventure, Drama]","In San Andreas, California is experiencing a s...","The film opens with the girl, Natalie (Morgan ...",6.1,2015-05-29
573384,False,tt0105643,1,2006-08-03,Quite probably the worst theatrically released...,"I don't even know what this was, it was that b...",ur4195782,1h 35min,"[Comedy, Fantasy, Horror]",A young child is terrified to discover that a ...,...,A young child is terrified to discover that a ...,,2.8,1990-10-12,1h 35min,"[Comedy, Fantasy, Horror]",A young child is terrified to discover that a ...,,2.8,1990-10-12


,is_spoiler,movie_id,rating_review,review_date,review_summary,review_text,user_id,duration_review,genre_review,plot_summary_review,...,plot_summary_movie,plot_synopsis_movie,rating_review,release_date_movie,duration,genre,plot_summary,plot_synopsis,rating_movie,release_date
573380,True,tt1276104,10,2017-03-30,"A well-acted, intriguing film.","Film noir meets sci-fi as great acting, though...",ur43650030,1h 53min,"[Action, Crime, Drama]","Joe is classified as a ""looper"", a job in whic...",...,"Joe is classified as a ""looper"", a job in whic...","In 2044, the United States has suffered econom...",7.4,2012-09-28,1h 53min,"[Action, Crime, Drama]","Joe is classified as a ""looper"", a job in whic...","In 2044, the United States has suffered econom...",7.4,2012-09-28
573381,False,tt0112471,8,2000-10-05,Once Again Linklater Avoids Mediocrity!,"As in his other movies, Linklater has a terrif...",ur0443283,1h 41min,"[Drama, Romance]",American tourist Jesse and French student Celi...,...,American tourist Jesse and French student Celi...,The film starts with Jesse meeting Céline on a...,8.1,1995-01-27,1h 41min,"[Drama, Romance]",American tourist Jesse and French student Celi...,The film starts with Jesse meeting Céline on a...,8.1,1995-01-27
573382,False,tt1013752,6,2014-01-14,A good reboot to the franchise.,The Fast and the Furious series got really wea...,ur22618683,1h 47min,"[Action, Crime, Thriller]",Heading back to the streets where it all began...,...,Heading back to the streets where it all began...,This fourth 'Fast and the Furious' film begins...,6.6,2009-04-03,1h 47min,"[Action, Crime, Thriller]",Heading back to the streets where it all began...,This fourth 'Fast and the Furious' film begins...,6.6,2009-04-03
573383,False,tt2126355,7,2015-05-29,Surprisingly Good Disaster Movie,I went in to this with admittedly very low exp...,ur4844673,1h 54min,"[Action, Adventure, Drama]","In San Andreas, California is experiencing a s...",...,"In San Andreas, California is experiencing a s...","The film opens with the girl, Natalie (Morgan ...",6.1,2015-05-29,1h 54min,"[Action, Adventure, Drama]","In San Andreas, California is experiencing a s...","The film opens with the girl, Natalie (Morgan ...",6.1,2015-05-29
573384,False,tt0105643,1,2006-08-03,Quite probably the worst theatrically released...,"I don't even know what this was, it was that b...",ur4195782,1h 35min,"[Comedy, Fantasy, Horror]",A young child is terrified to discover that a ...,...,A young child is terrified to discover that a ...,,2.8,1990-10-12,1h 35min,"[Comedy, Fantasy, Horror]",A young child is terrified to discover that a ...,,2.8,1990-10-12
